In [1]:
import os
from crewai import Agent
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import tool
from crewai import Task
from crewai import Crew
import yfinance as yf
from env import settings

#API_KEY
os.environ["OPENAI_API_KEY"] = settings.DATABASE_CONFIG['OPEN_API_KEY']


@tool("Stock News")
def stock_news(ticker):
    """
    Useful to get news about a stock. 
    The input should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.news

@tool("Stock Price")
def stock_price(ticker):
    """
    Useful to get stock price data. 
    The input should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.history(period="1mo")

@tool("Income Statement")
def income_stmt(ticker):
    """
    Useful to get the income statement of a company. 
    The input should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.income_stmt

@tool("Balance Sheet")
def balance_sheet(ticker):
    """
    Useful to get the balance sheet of a company. 
    The input should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.balance_sheet

@tool("Insider Transactions")
def insider_transactions(ticker):
    """
    Useful to get insider transactions of a stock. 
    The input should be a ticker, for example AAPL, NET.
    """
    ticker = yf.Ticker(ticker)
    return ticker.insider_transactions

scrape_tool = ScrapeWebsiteTool()


#Agent
researcher = Agent(
    role = "Researcher",
    goal = """
    Gather and interpret vast amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.
    """,
    backstory = """
    You're skilled in gathering and interpreting data from various sources. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.
    """,
    tools = [
        scrape_tool,
        stock_news,    
    ],
)

technical_analyst = Agent(
    role = "Technical Analyst",
    goal = """
    Analyze the movements of a stock and provide insights on trends, entry points, resistance and support levels.
    """,
    backstory = """
    An expert in technical analysis, you're known for your ability to predict stock prices. You provide valuable insights to your customers.
    """,
    tools = [
        stock_price,
    ]
)

financial_analyst = Agent(
    role = "Financial Analyst",
    goal = """
    Use financial statements, insider trading data and other metrics to evaluate a stock's financial health and performance.
    """,
    backstory = """
    You're a very experienced investment advisor that looks at a company's financial health, market sentiment, and qualitative data to make informed recommendations.
    """,
    tools = [
        income_stmt,
        balance_sheet,
        insider_transactions,
    ]
)

hedge_fund_manager = Agent(
    role = "Hedge Fund Manager",
    goal = """
    Manage a portfolio of stocks and make investment decisions to maximize returns using insights from financial analysts and researchers.
    """,
    backstory = """
    You're a seasoned hedge fund manager with a proven track record of making profitable investments. You always impress your clients.
    """,
    verbose = True,
)


# Task
research = Task(
    description = """
    Gather and analyze the latest news and market sentiment surrounding {company}'s stock. Provide a summary of the news and any notable shifts in sentiment.
    """,
    agent = researcher,
    expected_output="""
    Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock.
    """,
)
technical_analysis = Task(
    description = """
    Conduct a technical analysis of the {company} stock price movements and identify key support and resistance levels chart patterns.
    """,
    agent = technical_analyst,
    expected_output="""
    Your final answer MUST be a report with potential entry points, price targets and any other relevant information.
    """,
)
financial_analysis = Task(
    description = """
    Analyze the {company}'s financial statements, balance sheet, insider trading data and other metrics to evaluate {company}'s financial health and performance.
    """,
    agent = financial_analyst,
    expected_output="""
    Your final answer MUST be a report with an overview an overview of {company}'s revenue, earnings, cash flow, and other key financial metrics.
    """,
)
investment_recommendation = Task(
    description = """
    Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company} stock.
    """,
    agent = hedge_fund_manager,
    expected_output="""
    Your final answer MUST be a detailed recommendation to BUY, SELL or HOLD the stock. Provide a clear rationale for your recommendation.
    """,
    context = [
        research,
        technical_analysis,
        financial_analysis,
    ],
    output_file = "{company}_investment_recommendation.md"
)


# Crew, Result
crew = Crew(
    tasks = [
        research,
        technical_analysis,
        financial_analysis,
        investment_recommendation
    ],
    agents = [
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs = {
        "company" : "NVIDIA"
    }
)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


 [2024-08-20 17:44:09][DEBUG]: == Working Agent: Researcher
 [2024-08-20 17:44:09][INFO]: == Starting Task: 
    Gather and analyze the latest news and market sentiment surrounding NVIDIA's stock. Provide a summary of the news and any notable shifts in sentiment.
    
 [2024-08-20 17:44:16][DEBUG]: == [Researcher] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 17:44:16][DEBUG]: == Working Agent: Technical Analyst
 [2024-08-20 17:44:16][INFO]: == Starting Task: 
    Conduct a technical analysis of the NVIDIA stock price movements and identify key support and resistance levels chart patterns.
    
 [2024-08-20 17:44:23][DEBUG]: == [Technical Analyst] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 17:44:23][DEBUG]: == Working Agent: Financial Analyst
 [2024-08-20 17:44:23][INFO]: == Starting Task: 
    Analyze the NVIDIA's financial statements, balance sheet, insider trading data and other metrics to evaluate NVIDIA's finan